# Generating new Objects w/ `Generative Adversarial Network (GAN)`

### Import dependencies

In [1]:
import os
import sys
import datetime as dt

import numpy as np
import tensorflow as tf

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

data_dir = 'datasets/MNIST/'
data = input_data.read_data_sets(data_dir, one_hot=True)

# from dataset import ImageDataset

# data_dir = 'datasets/101_ObjectCategories/'
# save_file = 'saved/data.pkl'

# data = ImageDataset(data_dir=data_dir, size=24, grayscale=True, flatten=True)
# data.create()
# data.save(save_file=save_file, force=True)
# # data = data.load(save_file=save_file)

Extracting datasets/MNIST/train-images-idx3-ubyte.gz
Extracting datasets/MNIST/train-labels-idx1-ubyte.gz
Extracting datasets/MNIST/t10k-images-idx3-ubyte.gz
Extracting datasets/MNIST/t10k-labels-idx1-ubyte.gz


### Hyperparameters

In [3]:
# Inputs
img_size = 28    # data.size
img_channel = 1  # data.channel
img_size_flat = img_size * img_size * img_channel
print(f'Images »»» Size: {img_size:,}\tChannel: {img_channel:,}\tFlattened: {img_size_flat:,}')

Images »»» Size: 28	Channel: 1	Flattened: 784


In [4]:
# Network
kernel_size = 5
n_noise     = 64
keep_prob   = 0.8

In [5]:
# Training
batch_size    = 24
learning_rate = .01
save_interval = 100
log_interval  = 1000
iterations    = 10000

### Helpers

In [6]:
import matplotlib.pyplot as plt

def visualize(imgs, name=None, smooth=False, **kwargs):
    # Plot images in grid
    grid = int(np.sqrt(len(imgs)))
    # Create figure with sub-plots.
    fig, axes = plt.subplots(grid, grid)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Interpolation type.
        interpolation = 'spline16' if smooth else 'nearest'
        shape = [img_size, img_size]
        ax.imshow(imgs[i].reshape(shape), interpolation=interpolation, **kwargs)
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    if name:
        plt.suptitle(name)
    plt.show()

In [9]:
def lrelu(x, alpha=0.2):
    return tf.maximum(x, tf.multiply(x, alpha))

In [10]:
def binary_cross_entropy(x, z, eps=1e-12):
    return (-(x * tf.log(z + eps) + (1. - x) * tf.log(1. - z + eps)))

### The Discriminator

In [15]:
def discriminator(X, reuse=None):
    with tf.variable_scope('discriminator', reuse=reuse):
        # reshape
        net = tf.reshape([-1, img_size, img_size, img_channel])
        # conv + dropout
        net = tf.layers.conv2d(net, filters=64, kernel_size=5, strides=2, padding='SAME', activation=lrelu)
        net = tf.nn.dropout(net, keep_prob=keep_prob)
        # conv + dropout
        net = tf.layers.conv2d(net, filters=64, kernel_size=5, strides=1, padding='SAME', activation=lrelu)
        net = tf.nn.dropout(net, keep_prob=keep_prob)
        # conv + dropout
        net = tf.layers.conv2d(net, filters=64, kernel_size=5, strides=1, padding='SAME', activation=lrelu)
        net = tf.nn.dropout(net, keep_prob=keep_prob)
        # flatten
        net = tf.contrib.layers.flatten(net)
        # 2 fully connected layers
        net = tf.layers.dense(net, units=128, activation=lrelu)
        net = tf.layers.dense(net, units=1, activation=tf.nn.sigmoid)
        return net

### The Generator

In [16]:
def generator(noise, is_training=False):
    decay = 0.99
    with tf.variable_scope('generator'):
        net = noise
        d1 = 4
        d2 = 1
        # fully connected + dropout + batch norm
        net = tf.layers.dense(net, units=d1*d1*d2, activation=lrelu)
        net = tf.nn.dropout(net, keep_prob=keep_prob)
        net = tf.contrib.layers.batch_norm(net, decay=decay, is_training=is_training)
        # reshape + resize
        net = tf.reshape(net, shape=[-1, d1, d1, d2])
        net = tf.image.resize_images(net, size=[7, 7])
        # conv transpose + dropout + batch_norm
        net = tf.layers.conv2d_transpose(net, filters=64, kernel_size=5, strides=2, padding='SAME', activation=lrelu)
        net = tf.nn.dropout(net, keep_prob=keep_prob)
        net = tf.contrib.layers.batch_norm(net, decay=decay, is_training=is_training)
        # conv transpose + dropout + batch_norm
        net = tf.layers.conv2d_transpose(net, filters=64, kernel_size=5, strides=2, padding='SAME', activation=lrelu)
        net = tf.nn.dropout(net, keep_prob=keep_prob)
        net = tf.contrib.layers.batch_norm(net, decay=decay, is_training=is_training)
        # conv transpose + dropout + batch_norm
        net = tf.layers.conv2d_transpose(net, filters=64, kernel_size=5, strides=1, padding='SAME', activation=lrelu)
        net = tf.nn.dropout(net, keep_prob=keep_prob)
        net = tf.contrib.layers.batch_norm(net, decay=decay, is_training=is_training)
        # conv transpose
        net = tf.layers.conv2d_transpose(net, filters=64, kernel_size=5, strides=1, padding='SAME', activation=tf.nn.sigmoid)
        return net